In [1]:
from google.colab import files
file = files.upload()

Saving Frankenstein.txt to Frankenstein.txt


In [2]:
f = open('Frankenstein.txt', 'r')

lines = []
for i in f:
  lines.append(i)
text = ' '.join(lines)
text = text.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“', '').replace('”','').replace("'", '').replace('.','').replace(',','').replace(':','').replace(';','')
text = text.split()
text = ' '.join(text)

In [3]:
from keras.preprocessing.text import Tokenizer
import pickle
import numpy as np

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
pickle.dump(tokenizer,open('tokenizer.pkl', 'wb'))

sequences = tokenizer.texts_to_sequences([text])[0]

Tri-gram based model

In [5]:
X = []
y = []

k = 3
n = len(sequences)
for i in range(0, n - k):
  X.append(sequences[i: i + k])
  y.append(sequences[i + k])

X = np.array(X)
y = np.array(y)

In [6]:
from tensorflow.keras.utils import to_categorical

In [7]:
vocab_size = max(y) + 1
y = to_categorical(y, num_classes = vocab_size)

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.optimizers import Adam

In [17]:
embed_dim = 256
lstm_out = 200
batch_size = 64

model = Sequential()
model.add(Embedding(vocab_size, embed_dim, input_length = 3))
model.add(LSTM(lstm_out, return_sequences=True))
model.add(LSTM(lstm_out))
model.add(Dense(1000, activation ='relu'))
model.add(Dense(vocab_size, activation = 'softmax'))

In [18]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 3, 256)            1928704   
                                                                 
 lstm_2 (LSTM)               (None, 3, 200)            365600    
                                                                 
 lstm_3 (LSTM)               (None, 200)               320800    
                                                                 
 dense_2 (Dense)             (None, 1000)              201000    
                                                                 
 dense_3 (Dense)             (None, 7534)              7541534   
                                                                 
Total params: 10,357,638
Trainable params: 10,357,638
Non-trainable params: 0
_________________________________________________________________


In [19]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('next_words.h5', monitor = 'loss', verbose = 1, save_best_only = True)

In [20]:
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X, y, epochs = 100, batch_size = batch_size, callbacks=[checkpoint])

Epoch 1/100
1222/1224 [============================>.] - ETA: 0s - loss: 6.5121 - accuracy: 0.0701
Epoch 1: loss improved from inf to 6.51158, saving model to next_words.h5
1224/1224 [==============================] - 15s 10ms/step - loss: 6.5116 - accuracy: 0.0701
Epoch 2/100
1224/1224 [==============================] - ETA: 0s - loss: 5.8673 - accuracy: 0.1208
Epoch 2: loss improved from 6.51158 to 5.86729, saving model to next_words.h5
1224/1224 [==============================] - 12s 10ms/step - loss: 5.8673 - accuracy: 0.1208
Epoch 3/100
1222/1224 [============================>.] - ETA: 0s - loss: 5.4315 - accuracy: 0.1476
Epoch 3: loss improved from 5.86729 to 5.43142, saving model to next_words.h5
1224/1224 [==============================] - 12s 10ms/step - loss: 5.4314 - accuracy: 0.1476
Epoch 4/100
1224/1224 [==============================] - ETA: 0s - loss: 5.0853 - accuracy: 0.1667
Epoch 4: loss improved from 5.43142 to 5.08529, saving model to next_words.h5
1224/1224 [======

In [36]:
from keras.models import load_model
model = load_model('next_words.h5')
tokenizer = pickle.load(open('tokenizer.pkl','rb'))

In [48]:
while True:
  text = input('Please enter sentence(Please write more than 2 words), 0 to exit: ')
  if text != '0':
    try:
      text = text.split(' ')
      text = text[-3: ]
      sequence = np.array(tokenizer.texts_to_sequences([text]))
      #print(model.predict(sequence)[0])
      output = np.argpartition(model.predict(sequence)[0], -5)[-5:]

      next_words = []
      for key, value in tokenizer.word_index.items():
        if value in output:
          next_words.append(key)
        if len(next_words) == 5:
          break
      print(next_words)
    except Exception as e:
      print(e)
  else:
    break

Please enter sentence(Please write more than 2 words), 0 to exit: I perused, for the first time
['the', 'i', 'saw', 'those', 'during']
Please enter sentence(Please write more than 2 words), 0 to exit: Six years have passed
['and', 'in', 'more', 'elizabeth', 'since']
Please enter sentence(Please write more than 2 words), 0 to exit: 0
